**FINAL PROJECT PART 3: Ranking**

- Iria Quintero (254373)
- Javier González (243078)
- Mireia Pou (251725)


# Imports, Drive and previous practices (necessary functions only)

## Part 0: Imports and Drive

In [ ]:
#!pip install wordcloud

In [13]:
import nltk
nltk.download('wordnet') # Download the wordnet dataset

from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import PorterStemmer

import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import pandas as pd
import numpy as np
import math
from numpy import linalg as la

from collections import Counter
import itertools
import networkx as nx
from datetime import datetime

from collections import defaultdict
import time
from array import array
import collections

import zipfile
import json

import csv
import string
import re
import math


from wordcloud import WordCloud
import matplotlib.pyplot as plt
import gensim
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import KeyedVectors
from gensim.models import Word2Vec

# Load the embedding model (BERT)
model = SentenceTransformer('all-MiniLM-L6-v2')  # Use a lightweight model for speed

# Load necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load a pre-trained NER model
nlp = spacy.load("en_core_web_sm")

#doc2vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import random

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Part 1: Data load, data processing

In [15]:
# Charging json data to variable data
# Specify name and path of .zip
zip_path = '/content/drive/Shareddrives/IRWA/Project/data/data.zip' #Change path to your own settings

# Open .zip
with zipfile.ZipFile(zip_path) as z:
    # Read JSON from .zip line by line
    with z.open('data/farmers-protest-tweets.json') as f:
        data = []  # Initialize an empty list to store the JSON objects
        for line in f:
            try:
                # Attempt to parse each line as a JSON object
                json_obj = json.loads(line)
                data.append(json_obj)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid line: {line} due to error: {e}")  # Print error and skip invalid lines


#Show JSON by printing a sample
print(f"Number of JSON objects loaded: {len(data)}")
if data:
    print("Sample JSON object:")
    print(data[0])

Number of JSON objects loaded: 117407
Sample JSON object:
{'url': 'https://twitter.com/ArjunSinghPanam/status/1364506249291784198', 'date': '2021-02-24T09:23:35+00:00', 'content': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF', 'renderedContent': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur twitter.com/ravisinghka/st…', 'id': 1364506249291784198, 'user': {'username': 'ArjunSinghPanam', 'displayname': 'Arjun Singh Panam', 'id': 45091142, 'description': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my

Function to read the doc id to tweet id map file

In [16]:
def load_docid_map_from_zip(zip_file_path, csv_file_name):
    docid_map = {}
    with zipfile.ZipFile(zip_file_path) as z:
        with z.open(csv_file_name) as file:
            reader = csv.DictReader(file.read().decode('utf-8').splitlines())  # Decode and read the lines
            for row in reader:
                docid_map[row['id']] = row['docId']  # Map 'id' to 'docId'
    return docid_map

In [17]:
docid_map = load_docid_map_from_zip(zip_path, 'data/tweet_document_ids_map.csv')

In [18]:
def build_terms(line):
    """
    Preprocess the text by removing stop words, stemming,
    transforming to lowercase, removing hashtags, and returning tokens.

    Argument:
    line -- string (text) to be preprocessed

    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))

    # Start preprocessing
    line = line.lower()  # Convert to lowercase

    #remove hashtags (before tokenization)
    line = re.sub(r'#\w+', '', line)

    line = re.sub(r'[^a-z0-9\s]', '', line)  # Remove every symbol that is not a letter or a number
    line = line.split()  # Tokenize the text to get a list of terms

    # Remove punctuation
    line = [word.translate(str.maketrans('', '', string.punctuation)) for word in line]
    line = [item for item in line if item not in stop_words]  # Remove stopwords
    line = [word for word in line if len(word) > 1]  # Remove single-letter words
    line = [stemmer.stem(word) for word in line]  # Stemming


    return line

Function to get DocID | TweetID | Tweet Content | Tokenized Tweet | Date | Hashtags | Likes | Retweets | Url from a json tweet

In [19]:
def extract_tweet_info_with_docid(tweet_json, docid_map):
    # Extract the same fields as before
    tweet_id = tweet_json.get('id', '')
    doc_id = docid_map.get(str(tweet_id))  # Use str(tweet_id) to ensure proper matching

    if not doc_id:
        return None

    original_content = tweet_json.get('content', '')
    tokenized_content = build_terms(original_content)  # Apply text preprocessing
    tweet_date = tweet_json.get('date', '')

    hashtags = re.findall(r'#\w+', original_content)
    hashtags = [re.sub(r'#', '', hashtag) for hashtag in hashtags]
    hashtags = [build_terms(hashtag) for hashtag in hashtags]
    hashtags = [item for sublist in hashtags for item in sublist]
    tweet_likes = tweet_json.get('likeCount', 0)
    tweet_retweets = tweet_json.get('retweetCount', 0)
    reply_count = tweet_json.get('replyCount', 0)
    tweet_url = tweet_json.get('url', '')

    # Extract user information (username and follower count)
    user_info = tweet_json.get('user', {})
    username = user_info.get('username', 'unknown')
    followers_count = user_info.get('followersCount', 0)

    # Return the extracted information as a dictionary
    return {
        'DocID': doc_id,
        'TweetID': tweet_id,
        'Original Tweet': original_content,
        'Tokenized Tweet': tokenized_content,
        'Date': tweet_date,
        'Username': username,           # Twitter username
        'FollowersCount': followers_count,  # Number of followers
        'Hashtags': hashtags,
        'Likes': tweet_likes,
        'Retweets': tweet_retweets,
        'ReplyCount': reply_count,
        'Url': tweet_url
    }

In [8]:
processed_data = []

for json_obj in data:
    # Create a copy of JSON original object
    processed_obj = json_obj.copy()  # Copia todos los campos del objeto JSON original

    # Check if content is in the object
    if 'content' in processed_obj:
        # Extraer información necesaria
        processed_obj = extract_tweet_info_with_docid(processed_obj, docid_map)

    # Add the processed object to the list if is not none
    if processed_obj is not None:
        processed_data.append(processed_obj)

# Shows a sample of the processed data
print(f"Number of JSON objects processed: {len(processed_data)}")
if processed_data:
    print("Sample processed data:")
    print(processed_data[0])

Number of JSON objects processed: 48429
Sample processed data:
{'DocID': 'doc_0', 'TweetID': 1364506249291784198, 'Original Tweet': 'The world progresses while the Indian police and Govt are still trying to take India back to the horrific past through its tyranny. \n\n@narendramodi @DelhiPolice Shame on you. \n\n#ModiDontSellFarmers \n#FarmersProtest \n#FreeNodeepKaur https://t.co/es3kn0IQAF', 'Tokenized Tweet': ['world', 'progress', 'indian', 'polic', 'govt', 'still', 'tri', 'take', 'india', 'back', 'horrif', 'past', 'tyranni', 'narendramodi', 'delhipolic', 'shame', 'httpstcoes3kn0iqaf'], 'Date': '2021-02-24T09:23:35+00:00', 'Username': 'ArjunSinghPanam', 'FollowersCount': 603, 'Hashtags': ['modidontsellfarm', 'farmersprotest', 'freenodeepkaur'], 'Likes': 0, 'Retweets': 0, 'ReplyCount': 0, 'Url': 'https://twitter.com/ArjunSinghPanam/status/1364506249291784198'}


# Part 3

## 3.1. Ranking

### 3.1.1. TF-IDF + cosine similarity

To rank with tf-idf, we first need to create the index and then proceed to rank.

In [20]:
def create_index_tfidf(processed_data):
    """
    Implement the inverted index and compute tf, df, and idf for a collection of tweets.

    Returns:
    index - the inverted index containing terms as keys and the corresponding
            list of documents these keys appear in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appears in
    idf - inverse document frequency of each term
    tweet_content_index - mapping of DocID to the original tweet content
    """

    index = defaultdict(list)
    tf = defaultdict(list)  # Term frequencies of terms in documents
    df = defaultdict(int)  # Document frequencies of terms in the corpus
    tweet_content_index = defaultdict(str)  # Index for original tweet content
    num_documents = len(processed_data)  # Total number of documents (tweets)

    for tweet in processed_data:
        doc_id = tweet['DocID']
        terms = tweet['Tokenized Tweet']  # List of terms (already tokenized)
        original_tweet = tweet['Original Tweet']

        ## Build the current tweet index for TF normalization and document frequency
        current_tweet_index = {}

        for position, term in enumerate(terms):  # Terms from the tokenized tweet
            if term not in current_tweet_index:
                current_tweet_index[term] = [doc_id, array('I', [position])] # this is an entry
            else:
                current_tweet_index[term][1].append(position)

        # Normalize term frequencies
        norm = 0
        for entry in current_tweet_index.values():
            norm += len(entry[1]) ** 2
        norm = math.sqrt(norm)

        # Calculate TF and DF weights
        for term, entry in current_tweet_index.items():
            tf[term].append(np.round(len(entry[1]) / norm, 4))  # TF calculation
            df[term] += 1  # Increment DF for current term

            # Merge the current tweet index with the main index
            index[term].append(entry)

    # Compute IDF following the formula
    idf = {}
    for term in df:
        idf[term] = np.round(np.log(float(num_documents) / df[term]), 4)

    return index, tf, df, idf

In [21]:
# Create index
start_time = time.time()
index, tf, df, idf = create_index_tfidf(processed_data)
print("Total time to create the TF-IDF index: {} seconds" .format(np.round(time.time() - start_time, 2)))

Total time to create the TF-IDF index: 9.63 seconds


Now we have the index and the tf-idf, we can proceed to ranking:

In [22]:
def rank_documents(terms, docs, index, idf, tf):
    """
    Perform the ranking of the results of a search based on the tf-idf weights

    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies

    Returns:
    Print the list of ranked documents
    """
    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)

    # Compute the norm for the query tf
    query_terms_count = collections.Counter(terms)
    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):
        if term not in index:
            continue

        # Compute tf*idf (normalize TF as done with documents)
        query_vector[termIndex]= query_terms_count[term]/query_norm * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, entry in enumerate(index[term]):
            doc, postings = entry

            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]

    # Calculate the score of each doc
    # Compute the cosine similarity between queyVector and each docVector:

    doc_scores=[[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items() ]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]

    #print document titles instead if document id's
    #result_docs=[ title_index[x] for x in result_docs ]

    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)

    return result_docs, doc_scores

In [23]:
# Necessary function search in the ranking, searches queries that contains ALL the terms
def search_tf_idf(query, index):
    """
    Output is the list of documents that contain all of the query terms.
    So, we will get the list of documents for each query term, and take the intersection of them.
    """
    query = build_terms(query)
    docs = None  # Start with None to allow initialization by the first term
    for term in query:
        try:
            # Store in term_docs the ids of the docs that contain "term"
            term_docs = set(posting[0] for posting in index[term])

            if docs is None:
                docs = term_docs  # Initialize with the first term's docs
            else:
                docs = docs.intersection(term_docs) # (AND operation)
        except KeyError:
            # If any term is not in the index, no documents contain all terms, return empty list
            return []

    # Convert docs to a list if it's not empty
    docs = list(docs)

    #Rank the resulting documents
    ranked_docs, doc_scores = rank_documents(query, docs, index, idf, tf)
    return ranked_docs, doc_scores


In [24]:
# Try the function!
print("Insert your query (i.e.: Disha Ravi):\n")
query = input()
docs, score = search_tf_idf(query, index)
top = 10

print("\n======================\nTop {} results out of {} ranked documents for the searched query:".format(top, len(docs)))
for d_id in docs[:top]:
    tweet_info = next((tweet for tweet in processed_data if tweet['DocID'] == d_id), None)

    if tweet_info:
        # Print the original content of the tweet
        print("\n----------------------\n")
        print("doc_id = {} \ndoc_content = {}".format(d_id, tweet_info['Original Tweet']))

Insert your query (i.e.: Disha Ravi):

Disha Ravi

Top 10 results out of 628 ranked documents for the searched query:

----------------------

doc_id = doc_35225 
doc_content = Free Disha Ravi 

#FarmersProtest  #IndiaBeingSilenced

----------------------

doc_id = doc_32205 
doc_content = #IndiaBeingSilenced 
#farmersprotest 
Disha ravi https://t.co/OVmvJy3d8V

----------------------

doc_id = doc_28152 
doc_content = I Stand With Disha Ravi 
#ReleaseDishaRavi 
#FarmersProtest

----------------------

doc_id = doc_15603 
doc_content = #StandWithDishaRavi
I stand with Disha Ravi
#FarmersProtest

----------------------

doc_id = doc_11995 
doc_content = #FarmersProtest. Disha Ravi. https://t.co/VbW9rYLUIa

----------------------

doc_id = doc_24045 
doc_content = #releasedisha #ReleaseDishaRavi
#DishaRavi #DishaAnnappaRavi
#JusticeForDisha
#JusticeForDisha
#FarmersProtest

Her Name Is Disha Ravi, Not Disha Ravi Joseph: Friends Rubbish Fake Claim

https://t.co/Gb9nkYpH0x

---------------

### 3.1.2. Our score + cosine similarity

In [25]:
def rank_documents_personalized_with_context(terms, docs, index, idf, tf, processed_data):
    """
    Rank documents by combining TF-IDF, popularity metrics, hashtag boost, and contextual similarity.

    Parameters:
    - terms: list of original query terms and expanded synonyms
    - docs: list of document IDs to rank
    - index: inverted index data structure
    - idf: inverse document frequencies
    - tf: term frequencies
    - processed_data: list of tweet data including popularity metrics
    - context_weight: weight for contextual similarity

    Returns:
    - Ranked documents and their scores.
    """
    # Initialize document vectors and query vector
    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)

    '''
    # Preprocess terms: separate original terms and synonyms
    original_terms = set(build_terms(" ".join(terms)))  # Assume first terms are original
    expanded_query_terms = expand_query_with_synonyms(" ".join(original_terms))
    '''

    # Create the query vector with weighted TF-IDF values
    query_terms_count = collections.Counter(terms)
    query_norm = la.norm([query_terms_count[term] for term in terms])

    # For every query term, obtain the tfidf for the query and document vectors
    for termIndex, term in enumerate(terms):
        if term not in index:
            continue
        query_vector[termIndex] = (query_terms_count[term] / query_norm * idf[term])

        # Update document vectors
        for doc_index, entry in enumerate(index[term]):
            doc, postings = entry
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]

    # Calculate cosine similarity and contextual similarity
    doc_scores = []
    query_embedding = model.encode(" ".join(terms))

    for doc, curDocVec in doc_vectors.items():
        cosine_sim = np.dot(curDocVec, query_vector)

        # Retrieve popularity metrics for the tweet
        tweet_info = next((tweet for tweet in processed_data if tweet['DocID'] == doc), None)
        if tweet_info:
            retweet_score = tweet_info['Retweets'] * 0.4
            like_score = tweet_info['Likes'] * 0.2
            comment_score = tweet_info.get('Comments', 0) * 0.4
            popularity_score = retweet_score + like_score + comment_score
        else:
            popularity_score = 0

        # Hashtag boost for original terms
        hashtag_boost = 1.2 if any(term in tweet_info['Hashtags'] for term in terms) else 1.0

        # Calculate contextual similarity
        doc_embedding = model.encode(tweet_info['Original Tweet'])
        contextual_similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]


        # Calculate the final score
        combined_score = (cosine_sim + popularity_score) * hashtag_boost + contextual_similarity
        doc_scores.append((combined_score, doc))

    # Sort by combined score in descending order
    doc_scores.sort(reverse=True, key=lambda x: x[0])
    ranked_docs = [x[1] for x in doc_scores]

    return ranked_docs, doc_scores


In [26]:
def search_personalized_with_context(query, index, idf, tf, processed_data):
    """
    Search for documents that contain all query terms and rank them based on personalized and contextual criteria.

    Parameters:
    - query: the search query string
    - index: inverted index data structure
    - idf: inverse document frequencies
    - tf: term frequencies
    - processed_data: list of tweet data including popularity metrics

    Returns:
    - Ranked documents and their scores.
    """
    # Query terms
    terms = build_terms(query)

    # Collect documents containing any expanded query terms
    docs = set()
    for term in terms:
        if term in index:
            term_docs = set(posting[0] for posting in index[term])
            docs.update(term_docs)

    # Rank the resulting documents using the personalized ranking with context
    ranked_docs, doc_scores = rank_documents_personalized_with_context(
        terms, list(docs), index, idf, tf, processed_data
    )
    return ranked_docs, doc_scores


In [27]:
# Try the personalized search function
print("Insert your query (e.g., 'Disha Ravi'):\n")
query = input()
docs, scores = search_personalized_with_context(query, index, idf, tf, processed_data)
top = 10

print("\n======================\nTop {} results out of {} ranked documents for the searched query:".format(top, len(docs)))
for rank, (score, doc_id) in enumerate(scores[:top], start=1):
    tweet_info = next((tweet for tweet in processed_data if tweet['DocID'] == doc_id), None)

    if tweet_info:
        # Print the rank, score, and original content of the tweet
        print("\n----------------------\n")
        print("Rank: {} | Score: {:.4f}".format(rank, score))
        print("doc_id = {} \ndoc_content = {}".format(doc_id, tweet_info['Original Tweet']))


Insert your query (e.g., 'Disha Ravi'):

Disha Ravi

Top 10 results out of 829 ranked documents for the searched query:

----------------------

Rank: 1 | Score: 3672.0499
doc_id = doc_38410 
doc_content = disha ravi, a 21-year-old climate activist, has been arrested by delhi police for sharing a toolkit @GretaThunberg  posted in support of the #farmersprotest. 

disha’s arrest is alarming and the world needs to pay attention. #freedisharavi

https://t.co/IYGsLpNjwZ

----------------------

Rank: 2 | Score: 3583.4351
doc_id = doc_38012 
doc_content = Disha Ravi broke down in court room and told judge that she had merely edited two lines in the toolkit and was supporting farmers. What is wrong in that? This is how this nation treats anyone who spreads awareness. #FarmersProtest

----------------------

Rank: 3 | Score: 1183.9029
doc_id = doc_37099 
doc_content = Disha Ravi is 21 yrs

A climate activist from India she campaigns for clean air, clean water and a liveable planet

She is now

### 3.1.3. BM25

In [28]:

# Define BM25 parameters
k1 = 1.2  # Controls term frequency scaling
b = 0.75  # Controls document length normalization

def rank_documents_bm25(query_terms, docs, index, idf, tf, avg_dl, doc_lengths):
    """
    Rank documents based on BM25 scoring.

    Parameters:
    - query_terms: list of terms in the query
    - docs: list of document IDs to rank
    - index: inverted index data structure
    - idf: dictionary of idf values for each term
    - tf: dictionary of term frequencies for each term in documents
    - avg_dl: average document length in the collection
    - doc_lengths: dictionary mapping each document ID to its length

    Returns:
    - bm25_scores: list of tuples (BM25 score, doc ID), sorted by score in descending order
    """
    bm25_scores = []

    # For every document, compute the BM25 components for every term and for every query term
    for doc in docs:
        score = 0
        dl = doc_lengths[doc]  # Length of the document
        K = k1 * ((1 - b) + b * (dl / avg_dl))  # Normalization factor for document length

        for term in query_terms:
            if term in index and doc in [d[0] for d in index[term]]:
                term_idf = idf[term]

                # Find term frequency for the current document in `tf`
                doc_index = [d[0] for d in index[term]].index(doc)
                term_tf = tf[term][doc_index]  # Extract term frequency for doc in the list

                # Calculate BM25 component for this term in the current document
                score += term_idf * ((term_tf * (k1 + 1)) / (term_tf + K))

        bm25_scores.append((score, doc))

    # Sort scores in descending order
    bm25_scores.sort(reverse=True, key=lambda x: x[0])
    return bm25_scores


In [29]:
def search_bm25(query, index, idf, tf, avg_dl, doc_lengths, processed_data):
    """
    Search for documents that contain all query terms and rank them using BM25.

    Parameters:
    - query: the search query string
    - index: inverted index data structure
    - idf: inverse document frequencies
    - tf: term frequencies
    - avg_dl: average document length in the collection
    - doc_lengths: dictionary mapping each document ID to its length
    - processed_data: list of tweet data

    Returns:
    - Ranked documents and their scores.
    """
    # Tokenize and preprocess the query to get terms
    query_terms = build_terms(query)
    docs = None  # Start with None to allow initialization by the first term

    # Find documents containing all query terms
    for term in query_terms:
        if term in index:
            term_docs = set(posting[0] for posting in index[term])
            docs = term_docs if docs is None else docs.intersection(term_docs)
        else:
            # If any term is not in the index, return an empty list (no matching docs)
            return []

    # If matching documents were found, rank them with BM25
    if docs:
        docs = list(docs)
        ranked_docs = rank_documents_bm25(query_terms, docs, index, idf, tf, avg_dl, doc_lengths)
        return ranked_docs
    else:
        return []


In [30]:
# Calculate document lengths and average document length
doc_lengths = {tweet['DocID']: len(tweet['Tokenized Tweet']) for tweet in processed_data}
avg_dl = sum(doc_lengths.values()) / len(doc_lengths)

# Try the BM25 search function
print("Insert your query (e.g., 'Disha Ravi'):\n")
query = input()
ranked_docs = search_bm25(query, index, idf, tf, avg_dl, doc_lengths, processed_data)
top = 10

print("\n======================\nTop {} results out of {} ranked documents for the searched query:".format(top, len(ranked_docs)))
for rank, (score, doc_id) in enumerate(ranked_docs[:top], start=1):
    tweet_info = next((tweet for tweet in processed_data if tweet['DocID'] == doc_id), None)

    if tweet_info:
        # Print the rank, score, and original content of the tweet
        print("\n----------------------\n")
        print("Rank: {} | Score: {:.4f}".format(rank, score))
        print("doc_id = {} \ndoc_content = {}".format(doc_id, tweet_info['Original Tweet']))


Insert your query (e.g., 'Disha Ravi'):

Increasing prices

Top 10 results out of 109 ranked documents for the searched query:

----------------------

Rank: 1 | Score: 7.4081
doc_id = doc_10847 
doc_content = Khalisathan is responsible to increase price of diesel and petrol 😀 #farmersProtest #MSPLawForAllCrops

----------------------

Rank: 2 | Score: 6.1407
doc_id = doc_33490 
doc_content = Earlier,when the prices on petrol were increased, the concerns of Bollywood actors used to increase, but now petrol prices are continuously increasing.#IndiaBeingSilenced #FarmersProtest #FarmersProstests #FarmersStandingFirm #DishaRavi #PetrolPriceHike #CowardBJP #mondaythoughts

----------------------

Rank: 3 | Score: 5.3448
doc_id = doc_27781 
doc_content = What if I told you that the constant increase in the petrol prices is to move the attention from farmer's protest?

#BJPdestroysDemocracy 
#FarmersProtest 
#FuelLoot 
#PetrolDieselPriceHike

----------------------

Rank: 4 | Score: 4.5215
d

## 3.2. Top-20 using word2vec + cosine similarity

###3.2.1. Word2Vec necessary functions

Train a Word2Vec model

In [31]:
# Prepare tokenized tweet data for training
tokenized_tweets = [tweet['Tokenized Tweet'] for tweet in processed_data]

# Train the Word2Vec model on the tokenized tweets
word2vec_model = Word2Vec(sentences=tokenized_tweets, vector_size=100, window=5, min_count=2, workers=4)

Compute the word2vec representation of our tweets

In [32]:
def compute_tweet_vector(tweet_tokens, model):
    """
    Generate a tweet vector by averaging word vectors.

    Parameters:
    - tweet_tokens: list of words (tokens) in the tweet
    - model: Word2Vec model

    Returns:
    - tweet_vector: Average vector of the words in the tweet
    """
    vectors = []
    for word in tweet_tokens:
        if word in model.wv.key_to_index:  # Check if the word is in the model's vocabulary
            vectors.append(model.wv[word])
    if vectors:
        tweet_vector = np.mean(vectors, axis=0)
    else:
        tweet_vector = np.zeros(model.vector_size)  # Return a zero vector if no words are in the model
    return tweet_vector

# Generate tweet vectors and store them in a dictionary for quick access
tweet_vectors = {tweet['DocID']: compute_tweet_vector(tweet['Tokenized Tweet'], word2vec_model) for tweet in processed_data}


rank

In [33]:
def rank_documents_word2vec(query_vector, tweet_vectors):
    """
    Rank documents based on cosine similarity with a given query vector.

    Parameters:
    - query_vector: numpy array representing the query
    - tweet_vectors: dictionary with tweet DocIDs as keys and tweet vectors as values

    Returns:
    - ranked_docs: list of tuples (cosine similarity, DocID) sorted by similarity in descending order
    """
    similarities = []
    for doc_id, tweet_vector in tweet_vectors.items():
        # Calculate cosine similarity using dot product and norms
        dot_product = np.dot(query_vector, tweet_vector)
        norm_query = np.linalg.norm(query_vector)
        norm_tweet = np.linalg.norm(tweet_vector)

        # Avoid division by zero
        if norm_query == 0 or norm_tweet == 0:
            similarity = 0.0
        else:
            similarity = dot_product / (norm_query * norm_tweet)

        similarities.append((similarity, doc_id))

    # Sort by similarity score in descending order
    ranked_docs = sorted(similarities, key=lambda x: x[0], reverse=True)
    return ranked_docs

search

In [34]:
def search_word2vec_manual(query, tweet_vectors, processed_data, model, index):
    """
    Search tweets based on cosine similarity using manually calculated dot product within the ranking function.

    Parameters:
    - query: search query string
    - tweet_vectors: dictionary of tweet vectors (computed in advance)
    - processed_data: list of tweet data
    - model: Word2Vec model
    - top_n: number of top results to return

    Returns:
    - top_results: list of tuples (cosine similarity, DocID)
    """
    # Tokenize and preprocess the query
    query_tokens = build_terms(query)

    #Filter documents that contain all query terms in the same way as tf-idf
    docs = None
    for term in query_tokens:
        try:
            term_docs = set(posting[0] for posting in index[term])

            if docs is None:
                docs = term_docs
            else:
                docs = docs.intersection(term_docs) # (AND operation)
        except KeyError:
            return []

    # Convert docs to a list if it's not empty
    docs = list(docs)

    #filter tweet vectors
    filtered_tweet_vectors = {doc_id: tweet_vectors[doc_id] for doc_id in docs if doc_id in tweet_vectors}
    #print("Filtered tweet vectors:", filtered_tweet_vectors)
    #print("Filtered tweet vectors length:", len(filtered_tweet_vectors))

    # Compute the query vector by averaging the vectors of query tokens
    query_vector = compute_tweet_vector(query_tokens, model)

    # Rank documents by similarity
    #print("Tweet vectors:", tweet_vectors)
    #print("Tweet vectors length:", len(tweet_vectors))
    ranked_docs = rank_documents_word2vec(query_vector, filtered_tweet_vectors)

    return ranked_docs


prompt to repeat for each query

In [35]:
def search_word2vec(query, tweet_vectors, processed_data, word2vec_model):
    # Assume the Word2Vec model and tweet vectors have already been generated
    # word2vec_model = trained Word2Vec model
    # tweet_vectors = dictionary of tweet DocIDs to their computed Word2Vec vectors

    # Prompt the user for a query and perform the search using Word2Vec-based cosine similarity

    # Use the Word2Vec-based search function to rank documents
    ranked_docs = search_word2vec_manual(query, tweet_vectors, processed_data, word2vec_model, index)

    # Display the top results
    top = 20
    print("\n======================\nTop {} results out of {} ranked documents for query: {}".format(top, len(ranked_docs), query))
    for rank, (score, doc_id) in enumerate(ranked_docs[:top], start=1):
        tweet_info = next((tweet for tweet in processed_data if tweet['DocID'] == doc_id), None)

        if tweet_info:
            # Print the rank, score, and original content of the tweet
            print("\n----------------------\n")
            print("Rank: {} | Score: {:.4f}".format(rank, score))
            print("doc_id = {} \ndoc_content = {}".format(doc_id, tweet_info['Original Tweet']))

###3.2.2 Test for our 5 queries

Query 1: India government farmers protest

In [36]:
search_word2vec('India government farmers protest', tweet_vectors, processed_data, word2vec_model)


Top 20 results out of 55 ranked documents for query: India government farmers protest

----------------------

Rank: 1 | Score: 0.9506
doc_id = doc_39097 
doc_content = To hear more about what led to widespread farmer protests in India &amp; what the government’s heavy-handed tactics for quashing dissent say about India’s future under Modi, listen to Worldly below @voxdotcom @jarielarvin

#FarmersProtest #शहीद_जवान_शहीद_किसान https://t.co/GxYjDjChtp

----------------------

Rank: 2 | Score: 0.9424
doc_id = doc_25562 
doc_content = “Farmers across India have peacefully organized and protested for months. Yet they have faced violence, persecution, and retaliation by the government.”  #FarmersProtest https://t.co/jOGalK7Y2P

----------------------

Rank: 3 | Score: 0.9417
doc_id = doc_8933 
doc_content = 87 US farmers’ unions have extended solidarity to the ongoing protests by farmers in India and called the protests “one of the world’s most vibrant protests in history.” But this arrogan

Query 2: Increasing prices

In [37]:
search_word2vec('Increasing prices', tweet_vectors, processed_data, word2vec_model)


Top 20 results out of 109 ranked documents for query: Increasing prices

----------------------

Rank: 1 | Score: 0.9834
doc_id = doc_10847 
doc_content = Khalisathan is responsible to increase price of diesel and petrol 😀 #farmersProtest #MSPLawForAllCrops

----------------------

Rank: 2 | Score: 0.9578
doc_id = doc_33490 
doc_content = Earlier,when the prices on petrol were increased, the concerns of Bollywood actors used to increase, but now petrol prices are continuously increasing.#IndiaBeingSilenced #FarmersProtest #FarmersProstests #FarmersStandingFirm #DishaRavi #PetrolPriceHike #CowardBJP #mondaythoughts

----------------------

Rank: 3 | Score: 0.9395
doc_id = doc_7119 
doc_content = Why the outcry of petrol price increases,   If There is a higher price in Mumbai, so go to Mizoram and fill it up like a farmer can sell his crop anywhere💁‍♂
#PetrolPriceHike
#FarmersProtest https://t.co/eXJPmlw29m

----------------------

Rank: 4 | Score: 0.9291
doc_id = doc_38875 
doc_content

Query 3: Narendra Modi

In [38]:
search_word2vec('Narendra Modi', tweet_vectors, processed_data, word2vec_model)


Top 20 results out of 69 ranked documents for query: Narendra Modi

----------------------

Rank: 1 | Score: 0.9029
doc_id = doc_9065 
doc_content = If prime minister narendra modi knows better whey’not he is teaching them?? #FarmersProtest https://t.co/uGwbsaFbA6

----------------------

Rank: 2 | Score: 0.8845
doc_id = doc_13190 
doc_content = Narendra Modi should answer why did Vijay Mallya, Nirav Modi , Mehul Chowksi and several private sector fraudster ran away with public money?
B4 preaching privatisation 😡

#modi #bjp #bjplies #bjpfails #FarmersProtest #AtmaNirbharBharat 

https://t.co/HMS5N1gWZz

----------------------

Rank: 3 | Score: 0.8685
doc_id = doc_26001 
doc_content = I dare you, amit shah and narendra modi to contest from punjab @smritiirani @SmritiIraniOffc 
#FarmersMakeIndia 
#FarmersProtest https://t.co/CpTZPjTpst

----------------------

Rank: 4 | Score: 0.8637
doc_id = doc_17153 
doc_content = I dislike Shri Narendra Modi, because his political ideology is "divi

Query 4: Disha Ravi

In [39]:
search_word2vec('Disha Ravi', tweet_vectors, processed_data, word2vec_model)


Top 20 results out of 628 ranked documents for query: Disha Ravi

----------------------

Rank: 1 | Score: 1.0000
doc_id = doc_32205 
doc_content = #IndiaBeingSilenced 
#farmersprotest 
Disha ravi https://t.co/OVmvJy3d8V

----------------------

Rank: 2 | Score: 1.0000
doc_id = doc_11995 
doc_content = #FarmersProtest. Disha Ravi. https://t.co/VbW9rYLUIa

----------------------

Rank: 3 | Score: 0.9996
doc_id = doc_35462 
doc_content = #FreeDishaRavi 
#FarmersProtest
Freezer Disha Ravi nowwww https://t.co/PP9KwfiVrD

----------------------

Rank: 4 | Score: 0.9985
doc_id = doc_10044 
doc_content = Hats off to Disha Ravi..

#FarmersProtest https://t.co/5QjLp5BVuQ

----------------------

Rank: 5 | Score: 0.9901
doc_id = doc_35814 
doc_content = Release Disha Ravi 

#शहीद_जवान_शहीद_किसान
#FarmersProtest https://t.co/KhaEkCejac

----------------------

Rank: 6 | Score: 0.9901
doc_id = doc_35520 
doc_content = Release Disha Ravi .

#शहीद_जवान_शहीद_किसान #DishaRavi #FarmersProtest https://

Query 5: Anime

In [40]:
search_word2vec('Anime', tweet_vectors, processed_data, word2vec_model)


Top 20 results out of 25 ranked documents for query: Anime

----------------------

Rank: 1 | Score: 0.9213
doc_id = doc_1565 
doc_content = The guy wearing blue turban is my spirit animal.

#FarmersProtest https://t.co/0iX9YaLNmW

----------------------

Rank: 2 | Score: 0.8546
doc_id = doc_23921 
doc_content = ‘He will win whose army is animated by the same spirit throughout all its ranks’ The art of war #SunTzu. It’s vital the leaders stand with all those falsely imprisoned and not throw anyone under the bus. #FreeDeepSidhu #FreeNodeepKaur #FreeDishaRavi #LakhaSidhana #FarmersProtest

----------------------

Rank: 3 | Score: 0.8525
doc_id = doc_38825 
doc_content = Unacceptable!!!   The fact that @DelhiPolice assumed beating an elderly man was normal and acceptable is disgraceful.   You animals have zero compassion or regard  for human life.  #farmersprotest https://t.co/1WdbhH1xVa

----------------------

Rank: 4 | Score: 0.8396
doc_id = doc_33877 
doc_content = Modi stray dogs ca

## 3.3. Better representation than word2vec?

###3.3.1. Trying Doc2Vec

In [41]:
# Sample 20% of the data - otherwise we have a memory error
sample_size = int(0.01 * len(processed_data))
sampled_data = random.sample(processed_data, sample_size)

# Create tagged data from the sample
tagged_data = [TaggedDocument(words=tweet['Tokenized Tweet'], tags=[tweet['TweetID']]) for tweet in sampled_data]

In [42]:
# Train the Doc2Vec model on the tokenized tweets
doc2vec_model = Doc2Vec(vector_size=10, window=3, min_count=2, workers=4, epochs=10)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)


MemoryError: 

Doc2Vec needs too much memory. Although, had we access to a huge source of memory, we believe that it could improve Word2Vec performance.

###3.3.2. Trying Sentence2Vec

Too much expensive to be used for practical purposes. Training is too slow. (explained in the report)

In [12]:
# Initialize SentenceTransformer model (using pre-trained model)
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# tokenized tweet data for training
tokenized_tweets = [tweet['Original Tweet'] for tweet in processed_data]

# tweet vectors by encoding entire tweets (sentences)
tweet_vectors = {tweet['TweetID']: sentence_model.encode(tweet['Original Tweet']) for tweet in processed_data}


KeyboardInterrupt: 

###3.3.3. GloVe

If you don't have *glove.6B.300d.txt* downloaded, execute commented cells below

In [43]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-11-14 15:13:42--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-11-14 15:13:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-11-14 15:13:42--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [45]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [46]:
glove_model = gensim.models.KeyedVectors.load_word2vec_format('glove.6B.300d.txt', no_header=True)


In [47]:
def compute_tweet_vector(tweet_tokens, glove_model):
    """
    Generate a tweet vector by averaging the word vectors.

    Parameters:
    - tweet_tokens: list of words (tokens) in the tweet
    - glove_model: pre-loaded GloVe word vectors

    Returns:
    - tweet_vector: Average vector of the words in the tweet
    """
    vectors = []
    for word in tweet_tokens:
        if word in glove_model:  # Check if the word exists in GloVe model
            vectors.append(glove_model[word])
    if vectors:
        tweet_vector = np.mean(vectors, axis=0)
    else:
        tweet_vector = np.zeros(glove_model['the'].shape)  # Use zeros vector if no words are in the model
    return tweet_vector

In [48]:
# Prepare tweet vectors for ranking
tweet_vectors = {tweet['DocID']:compute_tweet_vector(tweet['Tokenized Tweet'], glove_model) for tweet in processed_data}

In [49]:
def rank_documents_glove(query_vector, tweet_vectors):
    """
    Rank documents (tweets) based on cosine similarity with the query vector.

    Parameters:
    - query_vector: numpy array representing the query
    - tweet_vectors: dictionary with tweet DocIDs as keys and tweet vectors as values

    Returns:
    - ranked_docs: list of tuples (cosine similarity, DocID) sorted by similarity in descending order
    """
    similarities = []
    for doc_id, tweet_vector in tweet_vectors.items():
        dot_product = np.dot(query_vector, tweet_vector)
        norm_query = np.linalg.norm(query_vector)
        norm_tweet = np.linalg.norm(tweet_vector)

        if norm_query == 0 or norm_tweet == 0:
            similarity = 0.0
        else:
            similarity = dot_product / (norm_query * norm_tweet)

        similarities.append((similarity, doc_id))

    ranked_docs = sorted(similarities, key=lambda x: x[0], reverse=True)
    return ranked_docs


In [50]:
def search_glove(query, tweet_data, glove_model):
    """
    Search for tweets based on GloVe embeddings by computing cosine similarity with the query.

    Parameters:
    - query: Search query string.
    - tweet_data: List of tweet data, each containing a 'DocID' and 'Original Tweet'.
    - glove_model: Pre-trained GloVe model.

    Returns:
    - ranked_docs: List of tuples (cosine similarity, DocID) sorted by similarity in descending order.
    """
    # Preprocess the query and compute the query vector
    query_tokens = build_terms(query)
    query_vector = compute_tweet_vector(query_tokens, glove_model)

    #Filter documents that contain all query terms in the same way as tf-idf
    docs = None
    for term in query_tokens:
        try:
            term_docs = set(posting[0] for posting in index[term])

            if docs is None:
                docs = term_docs
            else:
                docs = docs.intersection(term_docs) # (AND operation)
        except KeyError:
            return []

    # Convert docs to a list if it's not empty
    docs = list(docs)
    filtered_tweet_vectors = {doc_id: tweet_vectors[doc_id] for doc_id in docs if doc_id in tweet_vectors}
    # Rank tweets based on similarity
    ranked_docs = rank_documents_glove(query_vector, filtered_tweet_vectors)
    return ranked_docs

In [51]:
print("Insert your query (e.g., 'Disha Ravi'):\n")
query = input()
ranked_docs = search_glove(query, tweet_vectors, glove_model)
top = 10

# Display the top results
top = 20
print("\n======================\nTop {} results out of {} ranked documents for query: {}".format(top, len(ranked_docs), query))
for rank, (score, doc_id) in enumerate(ranked_docs[:top], start=1):
    tweet_info = next((tweet for tweet in processed_data if tweet['DocID'] == doc_id), None)

    if tweet_info:
        # Print the rank, score, and original content of the tweet
        print("\n----------------------\n")
        print("Rank: {} | Score: {:.4f}".format(rank, score))
        print("doc_id = {} \ndoc_content = {}".format(doc_id, tweet_info['Original Tweet']))

Insert your query (e.g., 'Disha Ravi'):

Disha Ravi

Top 20 results out of 628 ranked documents for query: Disha Ravi

----------------------

Rank: 1 | Score: 1.0000
doc_id = doc_34928 
doc_content = In solidarity with Disha Ravi🙏
#Shame
#FarmersProtest
#IndiaBeingSilenced 
#DishaRavi https://t.co/CB2OzxbCy0

----------------------

Rank: 2 | Score: 1.0000
doc_id = doc_36255 
doc_content = #JusticeForDisha
Justice For Disha Ravi 
#FarmersProtest https://t.co/juR4gPo496

----------------------

Rank: 3 | Score: 1.0000
doc_id = doc_35814 
doc_content = Release Disha Ravi 

#शहीद_जवान_शहीद_किसान
#FarmersProtest https://t.co/KhaEkCejac

----------------------

Rank: 4 | Score: 1.0000
doc_id = doc_33873 
doc_content = @GretaThunberg #StandWithFarmers #FarmersProtest @PMOIndia, @nstomar RELEASE DISHA RAVI

----------------------

Rank: 5 | Score: 1.0000
doc_id = doc_35520 
doc_content = Release Disha Ravi .

#शहीद_जवान_शहीद_किसान #DishaRavi #FarmersProtest https://t.co/43giAGdkdt

---------